<a href="https://colab.research.google.com/github/0x1beef/uap/blob/main/nb/gimbal_object_data.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a href="https://kaggle.com/kernels/welcome?src=https://github.com/0x1beef/uap/blob/main/nb/gimbal_object_data.ipynb">
    <img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"   />
</a>

In [ ]:
url = 'https://raw.githubusercontent.com/0x1beef/uap/main/nb'
import urllib.request
for py_file in ['utils.py','common.py']:
    urllib.request.urlretrieve(f'{url}/{py_file}', py_file)
import utils, common

## **Get the frames**

In [ ]:
gimbal = common.gimbal_from_huggingface()

## **Match the last WH frame to the first BH frame**

In [ ]:
import matplotlib.pyplot as plt
def show(*imgs):
    for img in imgs:
        plt.imshow(img, cmap='gray', vmin=0, vmax=255)
        plt.show()

In [ ]:
white_hot = gimbal.get_frame(370)
black_hot = gimbal.get_frame(372)
inv_wh = ~white_hot
show(white_hot, inv_wh, black_hot)

def is_black_hot(frame):
    return frame >= 372

In [ ]:
import numpy as np
from scipy.optimize import curve_fit

def get_wh_to_bh_lookup():
    rows = slice(166,250); cols = slice(201, 228)
    inv_wh_roi = inv_wh[rows, cols]
    bh_roi = black_hot[rows, cols]
    show(inv_wh_roi, bh_roi)

    xdata = inv_wh_roi.ravel()
    ydata = bh_roi.ravel()
    def func(x, g, a, b):
        return a * np.power(x / 255, g) * 255 + b
    popt, pcov = curve_fit(func, xdata, ydata)
    (g,a,b) = popt
    print(f'(g,a,b) = ({g:.3f}, {a:.3f}, {b:.3f})')

    def func_gab(x):
        return func(x, g, a, b)
    gab_lookup = np.array([np.clip(func_gab(x),0,255) for x in range(0,256)], np.uint8)
    print(gab_lookup)

    show(np.take(gab_lookup, inv_wh_roi))
    show(np.take(gab_lookup, inv_wh))
    show(black_hot)
    #show(abs(func_gab(inv_wh)-black_hot))
    return gab_lookup

gab_lookup = get_wh_to_bh_lookup()
def make_black_hot(img):
    return np.take(gab_lookup, ~img)

## **Calculate the geometry**

In [ ]:
import matplotlib.pyplot as plt
import math
from dataclasses import dataclass
import cv2

class Roi:
    def __init__(self, full_img_shape):
        (h, w) = full_img_shape
        roi_factor = 12
        (ofsy, ofsx) = (int(h / roi_factor), int(w / roi_factor))
        (midy, midx) = (int(h / 2), int(w / 2))
        self.y0 = midy - ofsy; self.y1 = midy + ofsy
        self.x0 = midx - ofsx; self.x1 = midx + ofsx
    def get(self, img):
        return img[self.y0:self.y1, self.x0:self.x1]
    
def point_from_roi(point, roi):
    return (point[0] + roi.y0, point[1] + roi.x0)
def point_to_roi(point, roi):
    return (point[0] - roi.y0, point[1] - roi.x0)

def get_frame_roi(frame):
    img = gimbal.get_frame(frame)
    roi = Roi(img.shape)
    roi_img = roi.get(img)
    return (roi, roi_img)

def get_binary_img(img, factor):
    (thresh, binary_img) = cv2.threshold(img, 255 * factor, 255, cv2.THRESH_BINARY)
    return binary_img

def get_contour(binary_img):
    (contours, img) = cv2.findContours(binary_img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    (h, w) = binary_img.shape
    area_threshold = h * w * 0.7
    def area_key(contour):
        area = cv2.contourArea(contour)
        return area if area < area_threshold else 0
    contour = max(contours, key=area_key)
    return contour

@dataclass
class GeometryData:
    glare_angle: float # degrees
    center_of_mass: (float, float) # (x,y)
    def to_dict(self):
        return {
            'glare_angle': self.glare_angle,
            'center_of_mass_x': self.center_of_mass[0],
            'center_of_mass_y': self.center_of_mass[1],
        }

def get_contour_geometry(contour, roi):
    m = cv2.moments(contour)
    elongation_angle = math.atan2(2 * m['mu11'], m['mu20'] - m['mu02']) / 2
    center_of_mass = (m['m10'] / m['m00'], m['m01'] / m['m00'])
    return GeometryData(math.degrees(elongation_angle), point_from_roi(center_of_mass, roi))

def show_geometry(geom, roi, roi_img, contour):
    roi_rgb = cv2.cvtColor(roi_img, cv2.COLOR_GRAY2RGB)
    roi_rgb = cv2.drawContours(roi_rgb, [contour], -1, (0,255,0), 1)
    (x, y) = geom.center_of_mass; (x, y) = (int(x), int(y))
    roi_rgb = cv2.drawMarker(roi_rgb, point_to_roi((x,y), roi), (255,0,0), cv2.MARKER_CROSS, 5)
    plt.imshow(roi_rgb)
    plt.show()
    
def get_geometry(frame, show_geom = False):
    (roi, roi_img) = get_frame_roi(frame)
    if not is_black_hot(frame):
        roi_img = make_black_hot(roi_img)
    binary_img = get_binary_img(roi_img, 0.2)
    contour = get_contour(binary_img)
    geom = get_contour_geometry(contour, roi)
    if show_geom == True:
        show_geometry(geom, roi, roi_img, contour)
    return geom

def test_geometry():
    frame = 433
    geom = get_geometry(frame, show_geom = True)
    print(geom)
    
test_geometry()

In [ ]:
import pandas as pd
df_geom = pd.DataFrame.from_records(
    [get_geometry(frame).to_dict() for frame in range(0, gimbal.get_frame_count())]
)
df_geom.glare_angle.plot(legend = True)
df_geom

## **Get the calculated horizon values from Sitrec**

In [ ]:
!git clone --depth 1 https://github.com/MickWest/sitrec.git
!cd sitrec && cp config.js.example config.js

In [ ]:
!curl -fsSL https://bun.sh/install | bash

import os
os.environ['PATH'] = "~/.bun/bin:" + os.environ['PATH']

In [ ]:
!cd sitrec && bun install

In [ ]:
%%writefile horizon.js
// note: the Sitrec internal API may change over time
import { initSitch } from "./sitrec/src/indexCommon.js"
import { get_real_horizon_angle_for_frame } from "./sitrec/src/JetStuff.js"
import { Sit } from "./sitrec/src/Globals.js"
import csv from "./sitrec/src/js/jquery.csv.js"
import fs from "node:fs"

await initSitch("gimbal", "SitGimbal.js")

var horizon_column = [["human_horizon"]]
for(let frame = 0; frame < Sit.frames; frame++) {
    const horizon = get_real_horizon_angle_for_frame(frame);
    horizon_column.push([horizon])
}

const csv_text = csv.fromArrays(horizon_column)
fs.writeFileSync("sitrec_data.csv", csv_text)

In [ ]:
!bun horizon.js

In [ ]:
import pandas as pd
df_sitrec = pd.read_csv("sitrec_data.csv")
df_sitrec

## **Collect all the data and upload it to Hugging Face**

In [ ]:
df = df_geom.join(df_sitrec)
df

In [ ]:
metadata = {
    'wh_to_bh_lookup' : gab_lookup.tolist(),
    'fps': gimbal.fps
}
utils.to_parquet_ext(df, 'object_data.parquet', 'gimbal', metadata)

In [ ]:
utils.upload_to_huggingface('object_data.parquet', 'logicbear/gimbal/data')